# Preparing dataset

In [1]:
from datasets import load_dataset, Dataset
# loading QA dataset about law practice
raw_datasets = load_dataset("ymoslem/Law-StackExchange")
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['question_title', 'score', 'link', 'license', 'question_body', 'question_id', 'answers', 'tags'],
        num_rows: 24370
    })
})

In [2]:
x = raw_datasets['train'][0]
print(x['question_title'])
print(x['license']) # CC BY-SA 4.0 or CC BY-SA 3.0 (skipping that)
print(x['question_body'])
print(x['answers'])
print(x['tags'])
print(x['link']) # follows pattern https://law.stackexchange.com/questions/{question_id}/{link_title} (link_title is useless, it's sometimes modified!)

Why is drunk driving causing accident punished so much worse than just drunk driving?
CC BY-SA 4.0
<p>When people drink and drive and then cause an accident especially where if someone dies they get years and years in prison but just the act of drunk driving is punished way more lenient.  Shouldn't the 2, drunk driving and drunk driving then causing accident be similarly punished?  I feel like a lot of times it's luck whether an accident happens.</p>

[{'answer_id': 94666, 'body': '<h3>Moral luck</h3>\n<p>You have raised the issue of <em>moral luck</em>, a long recognized problem in criminal theory. The classic expositions of this issue are by <a href="https://en.m.wikipedia.org/wiki/Thomas_Nagel" rel="noreferrer">Thomas Nagel</a>, in his chapter, &quot;<a href="https://rintintin.colorado.edu/%7Evancecd/phil1100/Nagel1.pdf" rel="noreferrer">Moral Luck</a>&quot; (1979) and <a href="https://en.m.wikipedia.org/wiki/Bernard_Williams" rel="noreferrer">Bernard Williams</a>, &quot;<a href="ht

In [3]:
def explode_answers(examples):
    """Explode answers and copy other features"""
    exploded = {k: [] for k in examples.keys()}
    for i, answers in enumerate(examples['answers']):
        num_copies = len(answers)
        exploded['answers'].extend(answers)
        for k in filter(lambda x: x != 'answers', exploded):
            for _ in range(num_copies):
                exploded[k].append(examples[k][i])
    return exploded
# csv alternative
# df = raw_datasets['train'].with_format("pandas")[:]
# Dataset.from_pandas(df.explode('answers', ignore_index=True))
answers_datasets = raw_datasets.map(explode_answers, batched=True)
answers_datasets


DatasetDict({
    train: Dataset({
        features: ['question_title', 'score', 'link', 'license', 'question_body', 'question_id', 'answers', 'tags'],
        num_rows: 37823
    })
})

In [4]:
answers_datasets['train'][0]

{'question_title': 'Why is drunk driving causing accident punished so much worse than just drunk driving?',
 'score': 23,
 'link': 'https://law.stackexchange.com/questions/94665/why-is-drunk-driving-causing-accident-punished-so-much-worse-than-just-drunk-dri',
 'license': 'CC BY-SA 4.0',
 'question_body': "<p>When people drink and drive and then cause an accident especially where if someone dies they get years and years in prison but just the act of drunk driving is punished way more lenient.  Shouldn't the 2, drunk driving and drunk driving then causing accident be similarly punished?  I feel like a lot of times it's luck whether an accident happens.</p>\n",
 'question_id': 94665,
 'answers': {'answer_id': 94666,
  'body': '<h3>Moral luck</h3>\n<p>You have raised the issue of <em>moral luck</em>, a long recognized problem in criminal theory. The classic expositions of this issue are by <a href="https://en.m.wikipedia.org/wiki/Thomas_Nagel" rel="noreferrer">Thomas Nagel</a>, in his cha

In [5]:
def combine_text(examples):
    """Combines text fields and extracts answer_scores"""
    return {'link': ['https://law.stackexchange.com/questions/' + str(id) for id in examples['question_id']], # more robust as the question_title could be modified by the time passes
            'text': [title + '\n' + body + '\n' + answers['body'] + '\n theme tags: ' + '; '.join(tags) \
                        for title, body, answers, tags in zip(examples['question_title'], examples['question_body'], examples['answers'], examples['tags'])],
            'answer_score': [answers['score'] for answers in examples['answers']],
            'question_score': examples['score']
            }
combined_datasets = answers_datasets.map(combine_text, batched=True)
combined_datasets

DatasetDict({
    train: Dataset({
        features: ['question_title', 'score', 'link', 'license', 'question_body', 'question_id', 'answers', 'tags', 'text', 'answer_score', 'question_score'],
        num_rows: 37823
    })
})

In [68]:
import html
from bs4 import BeautifulSoup, MarkupResemblesLocatorWarning
import warnings

warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)

def html_to_text(html_string):
    unescaped = html.unescape(html_string)  # Handle entities like &amp;
    soup = BeautifulSoup(unescaped, 'html.parser')
    return soup.get_text()

def clean_text(examples):
    """Deals with HTML character codes in texts"""
    return {'text': [html_to_text(text) for text in examples['text']]}
clean_datasets = combined_datasets.map(clean_text, batched=True, remove_columns=['license', 'question_id'])
clean_datasets

DatasetDict({
    train: Dataset({
        features: ['question_title', 'score', 'link', 'question_body', 'answers', 'tags', 'text', 'answer_score', 'question_score'],
        num_rows: 37823
    })
})

# Calculating Embeddings

In [8]:
import torch
import os
import numpy as np
import random
CUDA_DEVICE_IDX = 4
os.environ["CUDA_VISIBLE_DEVICES"] = f"{CUDA_DEVICE_IDX}"
device_map = f"cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_map)

def set_seed(random_seed: int = 42):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

In [9]:
from transformers import AutoModel, AutoTokenizer
checkpoint = 'sentence-transformers/multi-qa-mpnet-base-dot-v1'
model = AutoModel.from_pretrained(checkpoint, device_map=device_map, torch_dtype="auto") # embedder
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [10]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embedding(documents, return_numpy=True):
    inputs = tokenizer(
        documents, padding=True, truncation=True, return_tensors="pt"
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}
    model_output = model(**inputs)
    embedding = cls_pooling(model_output)
    return embedding.detach().cpu().numpy() if return_numpy else embedding

In [11]:
embedding = get_embedding(clean_datasets['train']["text"][:10])
embedding.shape

(10, 768)

In [ ]:
# embedding_dataset = clean_datasets.map(lambda examples: {'embedding': get_embedding(examples['text'], return_numpy=True)}, batched=True, batch_size=16)['train'] # high batch_size leads to melting gpu memory
# embedding_dataset.save_to_disk('embedding')
# embedding_dataset

Saving the dataset (0/1 shards):   0%|          | 0/37823 [00:00<?, ? examples/s]

Dataset({
    features: ['question_title', 'score', 'link', 'question_body', 'answers', 'tags', 'text', 'answer_score', 'question_score', 'embedding'],
    num_rows: 37823
})

In [18]:
# since embedding calculation takes a bunch of time, let's download it
from datasets import load_from_disk
embedding_dataset = load_from_disk('embedding')

# [FAISS](https://faiss.ai/index.html)
This is Facebook's library for efficient search in a set of embeddings. What is awesome in it, is that it works even if vectors don't fit in RAM  \
Here the **CPU** version of this library will be used *(note: faiss-gpu is officially supported only with conda)*

In [19]:
import faiss
embedding_dataset.add_faiss_index(column="embedding", metric_type=faiss.METRIC_INNER_PRODUCT) # the chosen model uses inner prod

  0%|          | 0/38 [00:00<?, ?it/s]

Dataset({
    features: ['question_title', 'score', 'link', 'question_body', 'answers', 'tags', 'text', 'answer_score', 'question_score', 'embedding'],
    num_rows: 37823
})

In [ ]:
import pandas as pd
def find_nearest(query: str, k_nearest: int=5):
    """Returns DataFrame of k-nearest documents sorted from best to worst"""
    query_embeddings = get_embedding(query, return_numpy=True)
    scores, documents = embedding_dataset.get_nearest_examples("embedding", query_embeddings, k=k_nearest)
    df = pd.DataFrame.from_dict(documents)
    df['faiss_scores'] = scores
    df.sort_values('faiss_scores', ascending=False, inplace=True)
    return df

# Let's test it out (Inference)

In [96]:
from termcolor import colored
import textwrap
import time

def beauty_find(*find_args, max_width: int = 100, max_snippet_len: int = 500, **find_kwargs):
    """Prints FAISS search results in a beautiful formatted waw"""
    # convert DataFrame to list of dicts for easier processing
    start_time = time.time()
    results = find_nearest(*find_args, **find_kwargs).to_dict('records')
    def to_plain_text(field):
        if isinstance(field, dict):
            for k in field:
                field[k] = to_plain_text(field[k])
        return html_to_text(field) if isinstance(field, str) else field
    results = [{k: to_plain_text(v) for k, v in record.items()} for record in results]
    response_time = time.time() - start_time
    
    print(colored(f"\n{'=' * 60}", 'blue', attrs=['bold']))
    print(colored(f"SEARCH RESULTS ({len(results)} found)", 'white', 'on_blue', attrs=['bold']))
    print(colored(f"SEARCH COMPLETED IN {response_time:.4f} SECONDS", 'white', 'on_green', attrs=['bold']))
    print(colored(f"{'=' * 60}\n", 'blue', attrs=['bold']))
    
    for i, res in enumerate(results, 1):
        # header with rank and similarity score
        print(colored(f"{'=' * 50}", 'blue'))
        print(colored(f"RESULT #{i} | Similarity: {res.get('faiss_scores', 0):.4f}", 'blue', attrs=['bold']))
        print(colored('=' * 50, 'blue'))
        
        # title and link
        title = res.get('question_title', 'No title')
        print(colored("Title:", 'yellow', attrs=['bold']) + f" {title}")
        
        if 'link' in res:
            print(colored("Link: ", 'yellow', attrs=['bold']) + colored(res['link'], 'cyan'))
        
        # scores
        scores = []
        if 'question_score' in res:
            scores.append(f"Question: {res['question_score']}")
        if 'answer_score' in res:
            scores.append(f"Answer: {res['answer_score']}")
        if scores:
            print(colored("Scores: ", 'yellow', attrs=['bold']) + " | ".join(map(lambda x: str(x) + '👍', scores)))
        
        # tags
        if 'tags' in res:
            tags = ", ".join(res['tags']) if isinstance(res['tags'], list) else res['tags']
            print(colored("Tags:", 'yellow', attrs=['bold']) + f" {tags}")
        
        # text snippets
        def print_snippet(label, text):
            if text and isinstance(text, str):
                snippet = textwrap.shorten(text, width=max_snippet_len, placeholder="...")
                wrapped = textwrap.fill(
                    f"{colored(label, 'yellow', attrs=['bold'])} {snippet}", 
                    width=max_width,
                    initial_indent='  ',
                    subsequent_indent='  '
                )
                print(wrapped)
        
        print_snippet("Question:", res.get('question_body'))
        print_snippet("Answer:", res.get('answers').get('body'))
        
        print(colored('-' * 50, 'blue'))

In [97]:
beauty_find('Racism')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0567 SECONDS

RESULT #1 | Similarity: 24.4931
Title: What is "racism"?
Link: https://law.stackexchange.com/questions/52808
Scores: Question: -3👍 | Answer: 4👍
Tags: equal-protection
  Question: The phrase "the best applicant should get the job" is considered racist. A
  college professor is suspended for refusing to give preferential treatment to black students.
  Quoting MLK famous "I have a dream that my four little children will one day live in a nation
  where they will not be judged by the color of their skin but by the content of their character"
  can get you accused of being a racist. People get fired as racists for saying explicitly color-
  blind things such as "all lives matter." On...
  Answer: The term "racism" is a political one, not a legal one, which refers to a
  belief. It is not illegal in the US, and it is a belief that is protected under the First
  Amendment. There is a legal construct, "discrimination on the basis of

In [98]:
beauty_find('I stole a car')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0497 SECONDS

RESULT #1 | Similarity: 25.5927
Title: Can you "steal" your own stolen car?
Link: https://law.stackexchange.com/questions/81600
Scores: Question: 6👍 | Answer: 8👍
Tags: united-states, theft, oregon
  Question: John Doe's car was stolen, along with a copy of the key. The theif then
  sells the car and key to someone else. John Doe is walking down a sidewalk, and sees his car in
  someone's driveway. He checks the VIN, and it is indeed the same car. He was carrying another copy
  of the key (only one of his two copies was stolen, and he never took this copy off his keychain),
  unlocks the car, takes the new "owner's" belongings out and sets them on the ground, and drives
  home in his car. He had never...
  Answer: There's a good answer in this thread by Trap_Door_Spiders Assuming the car
  was really stolen, not "stolen" in the sense my friend borrowed it and never gave it back, you
  could absolutely recover the car. The rea

In [99]:
beauty_find('I got fired illegaly')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0506 SECONDS

RESULT #1 | Similarity: 24.2232
Title: US laws of firing an employee
Link: https://law.stackexchange.com/questions/26453
Scores: Question: 8👍 | Answer: 29👍
Tags: united-states, contract-law, employment, texas
  Question: This is a follow-up from my previous question. Following my discussion with
  HR about the ethical violation of my manager and some colleagues, I was suddenly fired without any
  prior notice from my current employer. They put a few false labels on me, and give me a
  termination letter. They also asked me to return the sign-on bonus back. What are the US laws for
  firing an employee? Shouldn't I get an unemployed payment for a few months or something similar?
  I'm in Texas, US. In the employment...
  Answer: At Will Employment - In General An "at will" employee in the U.S. can be
  fired at any time for any reason without any prior notice or warning. Outside a union shop or a
  civil service employment si

In [100]:
beauty_find('How to avoid paying taxes?')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0497 SECONDS

RESULT #1 | Similarity: 24.0097
Title: How do billionaires get out of paying taxes? Can "normal people" do the same thing?
Link: https://law.stackexchange.com/questions/81407
Scores: Question: -2👍 | Answer: 4👍
Tags: united-states, tax-law, income-tax
  Question: How do billionaires avoid having to pay taxes? Can "normal" people with
  average incomes do the same thing?
  Answer: By putting their finances into a complicated legal structure, moving assets
  and income through various jurisdictions, and lobbying to make that legal. They also employ highly
  skilled lawyers who understand the tax code and possible loopholes very well. I'm pretty sure that
  a "normal" person with average income could reduce their taxes to close to zero by hiring
  specialist lawyers, but the cost of those lawyers would exceed the tax savings. For a billionaire,
  the ratio between legal costs and...
----------------------------------------------

In [101]:
beauty_find('What if someone kicked an officer?')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0549 SECONDS

RESULT #1 | Similarity: 25.4816
Title: Would you be convicted of assault, etc. for defending the passenger in the United incident?
Link: https://law.stackexchange.com/questions/18414
Scores: Question: 4👍 | Answer: 3👍
Tags: united-states, criminal-law, police, assault, self-defense
  Question: I'm wondering what would have happened if one of the passengers who was
  witnessing the United incident unfold had stepped in and tried to defend the doctor, by force.
  Note that I'm assuming the circumstances are exactly as they played out here; for example, the
  officer (if he was really a police officer) is not obviously dressed as one, so you'd have to make
  your own inference as to whether he was a police officer. Specifically, I'm wondering what would
  happen if someone saw the doctor was...
  Answer: I suspect that you would not be convicted in the present case, because the
  jury would be sympathetic to the plight of the pe

In [102]:
beauty_find('What is considered to be self-defence?')


SEARCH RESULTS (5 found)
SEARCH COMPLETED IN 0.0543 SECONDS

RESULT #1 | Similarity: 26.0440
Title: Self defense after provocation on the part of the defender
Link: https://law.stackexchange.com/questions/48532
Scores: Question: 2👍 | Answer: 1👍
Tags: canada, self-defense
  Question: A friend of mine in university wants to ensure he can defend himself
  legally. In a library a group of individuals were talking enough to annoy him and as the library
  has explicit no talking policy he proceeded to tell the group of indivudals to shut the f up and
  after some brief exchange of words he gave them the finger and walked off. Clearly one individual
  from the group was verbally aggressive back. After said friend went to sit down he was glared at
  by said individual in an aggressive...
  Answer: Canada's law on self-defense can be found here. The force used must be
  "reasonable in the circumstances" and "the person's role in the incident" is listed as a factor to
  be considered. I think t